##### Multi-Source RAG

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# set the environment variables
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

Tools are utilities designed to be called by a model: their inputs are designed to be generated by models, and their outputs are designed to be passed back to models.

Wikipedia Tool : To search about the topic on Wikipedia

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [4]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=2000)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
print(wiki_tool.invoke({"query":"Rohit Sharma"}))

Page: Rohit Sharma
Summary: Rohit Sharma (born 30 April 1987) is an Indian international cricketer who currently plays for and captains the India national cricket team in Test and ODI matches. He previously captained in T20 matches and in the country's 2024 T20 World Cup win after which he retired from T20s in June. He is considered as one of the best batsmen of his generation and one of the greatest white-ball opening batters of all time. 
He holds several batting records which famously include most runs in T20 Internationals, most sixes in international cricket, most double centuries in ODI cricket (3), most centuries at Cricket World Cups (7) and joint most hundreds in Twenty20 Internationals (5). He is the first player to score 5 T20I centuries. He also holds the world record for the highest individual score (264) in a One Day International (ODI) match and is the only player to have scored three double-centuries in ODIs and also holds the record for scoring most hundreds (five) in 

In [6]:
print(f"Name: {wiki_tool.name}")
print(f"Description: {wiki_tool.description}")
print(f"args schema: {wiki_tool.args}")
print(f"returns directly?: {wiki_tool.return_direct}")

Name: wikipedia
Description: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
args schema: {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}
returns directly?: False


Reteiver tool : Creating tool to retrieve information about langsmith, store it in the database

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",model_kwargs={'device': 'cpu'},encode_kwargs={'normalize_embeddings': False})

chroma_db = Chroma('chroma_vectorstore',embedding_function=embedding)
chroma_db.add_documents(documents=documents,embedding=embedding)

retriever=chroma_db.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\admin\anaconda3\envs\env_langchain_tutorial\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever=retriever,
                                     name="langsmith_search",
                                     description=
                                     """
                                     Search for information about LangSmith. For any questions about LangSmith, you must use this tool!
                                     Search in the vector database.
                                     """)

Arvix Tool : Create a tool which search the arvix for specific research paper or related thing

In [24]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=3,doc_content_chars_max=500)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [25]:
arxiv_tool.invoke({'query':"Give me the summary of the attention all you need research paper"})

'Published: 2023-09-19\nTitle: Attention Is Not All You Need Anymore\nAuthors: Zhe Chen\nSummary: In recent years, the popular Transformer architecture has achieved great\nsuccess in many application areas, including natural language processing and\ncomputer vision. Many existing works aim to reduce the computational and memory\ncomplexity of the self-attention mechanism in the Transformer by trading off\nperformance. However, performance is key for the continuing success of the\nTransformer. In this pap'

In [26]:
tools = [arxiv_tool,wiki_tool,retriever_tool]

In [27]:
# model
from langchain_groq import ChatGroq
model_id = "llama-3.3-70b-versatile"
model = ChatGroq(
    model=model_id,
    temperature=0.5,
    max_retries=2,
)

Agents

In [28]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

c:\Users\admin\anaconda3\envs\env_langchain_tutorial\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [29]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [30]:
template= prompt.template
print(template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [31]:
model_with_tools = model.bind_tools(tools=tools)

In [32]:
model_with_tools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021B83E68190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021B83E998D0>, model_name='llama-3.3-70b-versatile', temperature=0.5, model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'arxiv', 'description': 'A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'wikipedia', 'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people,

In [33]:
from langchain_core.messages import HumanMessage
from langchain.agents import create_react_agent,AgentExecutor

In [20]:
agent = create_react_agent(llm=model,tools=tools,prompt=prompt)
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor.invoke({"input":"Rohit Sharma"})



> Entering new AgentExecutor chain...
To find information about Rohit Sharma, I should first consider what type of information is likely to be available about him. Rohit Sharma is a well-known cricketer, so I expect to find general information about his career, personal life, and achievements.

Action: wikipedia
Action Input: Rohit SharmaPage: Rohit Sharma
Summary: Rohit Sharma (born 30 April 1987) is an Indian international cricketer who currently plays for and captains the India national cricket team in Test and ODI matches. He previously captained in T20 matches and in the country's 2024 T20 World Cup win after which he retired from T20s in June. He is considered as one of the best batsmen of his generation and one of the greatest white-ball opening batters of all time. 
He holds several batting records which famously include most runs in T20 Internationals, most sixes in international cricket, most double centuries in ODI cricket (3), most centuries at Cricket World Cups (7) and 

{'input': 'Rohit Sharma',
 'output': "Rohit Sharma is an Indian international cricketer who captains the India national cricket team in Test and ODI matches. He is considered one of the best batsmen of his generation and has set several batting records, including most runs in T20 Internationals, most sixes in international cricket, and most double centuries in ODI cricket. He has won numerous awards, including the ICC Men's ODI Cricketer of the Year award in 2019, and has led the Mumbai Indians to five Indian Premier League titles. He is also a two-time T20 World Cup winner, having won the tournament in 2007 and 2024."}